# Workshop: Train a Medical AI Model in One Day
## Notebook 3: Visualizing 2 Model's results in fiftyone

Load local predictions

In [ ]:
%pip install fiftyone

In [ ]:
import fiftyone as fo

In [ ]:
# 🔧 Configuration
datasetUUID = "068818885c547b1e80002649e95254ce"
fiftyoneExportPath = "./.cache/voxel51/034cb8e37f5444e98a78f1be65fd0bff/068818885c547b1e80002649e95254ce/fiftyone_dataset"

In [ ]:
name = f"eyepop_dataset_{datasetUUID}"
# Check if the dataset exists
if name in fo.list_datasets():
    print(f"Dataset '{name}' exists. Loading...")
    dataset = fo.load_dataset(name)
else:
    print(f"Dataset '{name}' does not exist. Creating a new one...")
    # Clone the dataset with a new name and make it persistent
    dataset = fo.Dataset.from_dir(
        dataset_dir=fiftyoneExportPath,
        dataset_type=fo.types.FiftyOneDataset,
        name=name,
        persistent=True
    )
      
async def run():
    
    
    print("Loading dataset into FiftyOne...")    
    session = fo.launch_app(dataset, browser=True, auto=False)    
    
await run()

In [ ]:
# Iterate through the samples and update labels
for sample in dataset:
    for detection in sample.ground_truth.detections:
        if detection.label.startswith("stenosis"):
            detection.label = "stenosis"
    sample.save()

print("Labels updated successfully!")

In [ ]:
# Grab a sample (first, specific index, or with a filter)
sample = dataset.first()  # or sample = dataset[42]

# Check if predictions_300 exists and print its structure
if sample.predictions_300 is not None:
    print("predictions_300 detections:")
    for det in sample.predictions_300.detections:
        print(det.label, det.bounding_box, det.confidence)
else:
    print("predictions_300 is None")

# Do the same for predictions_3k
if sample.predictions_3k is not None:
    print("\npredictions_3k detections:")
    for det in sample.predictions_3k.detections:
        print(det.label, det.bounding_box, det.confidence)
else:
    print("predictions_3k is None")

In [ ]:
# Iterate over samples and fix labels
for sample in dataset:
    predictions = sample.predictions_300
    if predictions is not None and predictions.detections is not None:
        for detection in predictions.detections:
            if detection.label.startswith("stenosis"):
                detection.label = "stenosis"
        sample.predictions_300 = predictions  # assign back if modified
        sample.save()

In [ ]:
# Iterate over samples and fix labels
for sample in dataset:
    predictions = sample.predictions_3k
    if predictions is not None and predictions.detections is not None:
        for detection in predictions.detections:
            if detection.label.startswith("stenosis"):
                detection.label = "stenosis"
        sample.predictions_3k = predictions  # assign back if modified
        sample.save()

In [ ]:
print(dataset)

In [ ]:
results = dataset.evaluate_detections(
    "predictions_300",
    gt_field="ground_truth",
    eval_key="predictions_300_eval",
    compute_mAP=True,  # Avoids needing confidence
)
#results.print_report()

In [ ]:
results2 = dataset.evaluate_detections(
    "predictions_3k",
    gt_field="ground_truth",
    eval_key="predictions_3k_eval",
    compute_mAP=True,  # Avoids needing confidence
)
#results.print_report()

In [ ]:
# Get the 10 most common classes in the dataset
counts = dataset.count_values("ground_truth.detections.label")
classes_top10 = sorted(counts, key=counts.get, reverse=True)

# Print a classification report for the top-10 classes
results.print_report(classes=classes_top10)
results2.print_report(classes=classes_top10)

In [ ]:
print(results.mAP())
print(results2.mAP())

In [ ]:
import webbrowser

webbrowser.open("http://localhost:5151/")